# Imports

In [2]:
# Data Management
import pandas as pd

# Functions

In [4]:
def trader_df_helper(initial_df, active_traders):
    '''
    Allows for subsetting of all traders in the card log to ones you want to trade with

    Args:
        initial_df = the dataframe of all pooled data across every trainer in the log, no specificity of any parameter
        active_traders = the set of traders you want to be included in the analysis of potential trades

    Returns:
        A dataframe that is a subset of all the data containing only traders the user wants to involve in this trade analysis
    '''

    # Keep the data of the cards
    keep_list = ["ID","Card","Rarity","Pack"]    

    # Add the columns of the active traders' card counts
    for trader in active_traders:
        keep_list.append(trader + " has")
        keep_list.append(trader + " extra")


    # Safer to copy, remove for memory optimization
    active_df = initial_df.copy()
    
    # Return dataframe that is a subset the initial dataframe with only the data of the active traders
    return active_df[keep_list]

In [5]:
def get_best_trader_helper(same_rarity_df, giving_trader, receiver_trader, kwargs):
    '''
    Generates a dataframe of which cards can actually be traded based on which trainers have cards tradeable for the card a single trader needs
    
    Identifies which cards a giving trader most benefits from trading with a receiving trader who has already establish a need for a card the the 
    giving trader has
    
    Args:
        same_rarity_df = subset dataframe of cards at the same rarity level of the needed card of the receiving trader
            -> stipulation of trading platform, traded cards must be of same rarity
        giving_trader = the trader giving a card to the receiver trader
        receiver_trader = the trader who is getting a card they need from the giving_trader
        kwargs =
            mutually_benefitial = T/F whether the giving trader needs to have zero of the card they are to receive in return from the receiver trader
            robust = T/F whether to identify an opportunity for trade or list out all possible trades for a given opportunity

    Returns:
        kwargs -> mutually_beneficial = 1:
            mutually_beneficial_df = dataframe containing only trades where the receiver can reciprocate with a card the giver doesn't already have
                -> can be returned as empty to indiciate that there aren't any mutually beneficial trades
        kwargs -> mutually_beneficial = 0:
            all_trades_df = dataframe containing all cards that the receiver can reciprocate the giver's trade with, permitted by the platforms rules
    '''
    # Expand the giving and receiving trader into column labels for both cards they have and have extras of (willing to trade)     
    giving_trader_has_extra = giving_trader + " has"
    giving_trader_extra = giving_trader + " extra"

    receiver_trader_has_extra = receiver_trader + " has"
    receiver_trader_extra = receiver_trader + " extra"


    
    # Remove other potential traders from the dataframe that are irrelevant to this specific pairing
    # Get all columns from the dataframe
    drop_cols = list(same_rarity_df.columns)
    # Establish which columns to keep
    keep_cols = ["ID", "Card", "Rarity", "Pack", giving_trader_has_extra, giving_trader_extra, receiver_trader_has_extra, receiver_trader_extra]

    # Take each keep column out of the list of columns to be dropped
    for col in keep_cols:
        drop_cols.remove(col)

    # Drop irrelevant columns from the dataframe
    same_rarity_df = same_rarity_df.drop(drop_cols, axis=1)


    
    # If looking for mutually beneficial trades only it will be identified in the kwargs
    if "mutually_beneficial" in kwargs and kwargs["mutually_beneficial"] == 1:
        # Copy the cleaned rarity dataframe for a search for mutually beneficial trades
        # Could safely remove because of scoping but safer to leave in place, remove for memory optimization though
        mutually_beneficial_df = same_rarity_df.copy()
    
        # Iterrate through all the cards in the dataframe and eliminate non mutually beneficial trades
        for sub_idx, sub_row in mutually_beneficial_df.iterrows():
            # Drop any potential trade card that the giver already has
            # or
            # Drop any potential trade card that the receiver does not have extras of
            if sub_row[giving_trader_has_extra] > 0 or sub_row[receiver_trader_extra] == 0:
                mutually_beneficial_df = mutually_beneficial_df.drop(sub_idx)
    
        # Drop additional columns to leave just most readable data
        mutually_beneficial_df = mutually_beneficial_df.drop([giving_trader_extra, receiver_trader_has_extra], axis=1)

        # If there are mutually beneficial trades for this card, announce it and return the dataframe containing them
        if not mutually_beneficial_df.empty:
            print("Mutually Beneficial Trade Found")
            return mutually_beneficial_df.sort_values(by=[giving_trader_has_extra, receiver_trader_extra], ascending=[True, False])
        else:
            # Else announce that there aren't any mutually beneficial trades
            print("No Mutually Beneficial Trade Found")
            # Return the empty df for logical operation interpretation in the parent function
            return mutually_beneficial_df
    else:
        # If trades don't need to be mutually beneficial return a dataframe of all possible trades sorted by which cards the giver has the least of
        # and within that which cards the receiver has the most of
        all_trades_df = same_rarity_df.drop([giving_trader_extra, receiver_trader_has_extra], axis=1).sort_values(by=[giving_trader_has_extra, receiver_trader_extra], ascending=[True, False])
        # Return all possible trades, i.e. where the receiver has extras to reciprocally trade
        return all_trades_df[all_trades_df[receiver_trader_extra] != 0]          

In [6]:
def identify_traders(initial_df, active_traders, **kwargs):
    sub_df = trader_df_helper(initial_df, active_traders)
    
    has_cols = [col for col in sub_df.columns if "has" in col]
    extra_cols = [col for col in sub_df.columns if "extra" in col]

    for idx, row in sub_df.iterrows():
        # If there are no extras up for grabs or if everyone already has the card, skip
        if row[extra_cols].sum() == 0 or not 0 in list(row[has_cols]):
            continue
        else:
            # 1/0 truth list for who has extra of this card; location corresponds to person list
            has_extra_list = list(row[extra_cols] != 0)

            # 1/0 truth list for who needs this card; location corresponds to person list
            need_list = list(row[has_cols] == 0)
        
            # Traded cards must be of same rarity
            # Get a subset of data of cards that can be traded for this card
            same_rarity_df = sub_df[sub_df["Rarity"] == row["Rarity"]]

            

            # Identify who needs the card
            for i in range(len(need_list)):
                # If need_list true, then this person needs the card
                if need_list[i] != 0:
                    receiver_trader = active_traders[i]
                        
                    # Print who needs card
                    print(str(receiver_trader) + " needs " + row["Card"])

                    # Identify who has the card
                    for i in range(len(has_extra_list)):
                
                        if has_extra_list[i] != 0:
                            print(str(active_traders[i]) + " has " + row["Card"])
                            
                            trade_df = get_best_trader_helper(same_rarity_df, active_traders[i], receiver_trader, kwargs)
                            
                            if "robust" in kwargs.keys() and kwargs["robust"] == 1:
                                if not trade_df.empty:
                                    print("They can trade: ")
                                    display(trade_df)
                            else:
                                continue

            print("---")
                        

# Data Import/Cleaning

In [8]:
# Read in the spreadsheet csv containing traders' catalogs
df = pd.read_csv("PokeTrade_ExampleData.csv")
# Drop empty separator columns
df = df.drop(["Unnamed: 6", "Unnamed: 9", "Unnamed: 12", "Unnamed: 15"], axis=1)
# Hard rename of columns from the spreadsheet
df = df.rename(columns={
    "Aidan": "Aidan has",
    "Action": "Aidan extra",
    "Megan": "Megan has",
    "Action.1": "Megan extra",
    "AJ": "AJ has",
    "Action.2": "AJ extra",
    "Roan": "Roan has",
    "Action.3": "Roan extra",
    "Abby": "Abby has",
    "Action.4": "Abby extra",
})

# Fill NaNs with 0's for people who left empty spaces in catalog
df = df.fillna(0)

# Drop any rows that don't correspond to a card
df = df[df["ID"] != ""]

# Function Running

In [10]:
identify_traders(df, ["Megan","Aidan"], mutually_beneficial=1, robust=1)

Megan needs Starmie
Aidan has Starmie
Mutually Beneficial Trade Found
They can trade: 


,ID,Card,Rarity,Pack,Megan extra,Aidan has
425,52.0,Magneton,2d,ST,1.0,0.0
477,104.0,Spiritomb,2d,ST,1.0,0.0
522,149.0,Lum Berry,2d,ST,1.0,0.0
525,152.0,Cynthia,2d,ST,1.0,0.0
528,155.0,Mars,2d,ST,1.0,0.0


---
Megan needs Magikarp
Aidan has Magikarp
Mutually Beneficial Trade Found
They can trade: 


,ID,Card,Rarity,Pack,Megan extra,Aidan has
514,141.0,Chatot,1d,ST,4.0,0.0
505,132.0,Starly,1d,ST,3.0,0.0
466,93.0,Hippopotas,1d,ST,1.0,0.0
508,135.0,Bidoof,1d,ST,1.0,0.0


---
Megan needs Omanyte
Aidan has Omanyte
Mutually Beneficial Trade Found
They can trade: 


...

---
Aidan needs Cynthia
Megan has Cynthia
Mutually Beneficial Trade Found
They can trade: 


,ID,Card,Rarity,Pack,Megan has,Aidan extra
74,75.0,Starmie,2d,GA,0.0,1.0
80,81.0,Omanyte,2d,GA,0.0,1.0
418,45.0,Abomasnow,2d,ST,0.0,1.0


---
Aidan needs Mars
Megan has Mars
Mutually Beneficial Trade Found
They can trade: 


,ID,Card,Rarity,Pack,Megan has,Aidan extra
74,75.0,Starmie,2d,GA,0.0,1.0
80,81.0,Omanyte,2d,GA,0.0,1.0
418,45.0,Abomasnow,2d,ST,0.0,1.0


---
Aidan needs Gastrodon
Megan has Gastrodon
No Mutually Beneficial Trade Found
---
Aidan needs Cresselia
Megan has Cresselia
No Mutually Beneficial Trade Found
---
Aidan needs Palkia ex
Megan has Palkia ex
No Mutually Beneficial Trade Found
---


In [11]:
pack_spec_df = df[df["Pack"].isin(["GA","MI"])]
identify_traders(pack_spec_df, ["Megan","Aidan","AJ"], mutually_beneficial=0, robust=1)

AJ needs Ivysaur
Aidan has Ivysaur
They can trade: 


,ID,Card,Rarity,Pack,Aidan has,AJ extra
225,226.0,Lt. Surge,2d,GA,0.0,4.0
37,38.0,Ninetales,2d,GA,1.0,3.0
111,112.0,Pincurchin,2d,GA,2.0,2.0
201,202.0,Chansey,2d,GA,2.0,1.0
160,161.0,Mienshao,2d,GA,3.0,1.0
199,200.0,Dodrio,2d,GA,3.0,1.0
51,52.0,Centiskorch,2d,GA,4.0,3.0
164,165.0,Arbok,2d,GA,4.0,3.0
25,26.0,Pinsir,2d,GA,4.0,2.0
42,43.0,Rapidash,2d,GA,4.0,2.0


---
AJ needs Kakuna
Megan has Kakuna
They can trade: 


,ID,Card,Rarity,Pack,Megan has,AJ extra
76,77.0,Magikarp,1d,GA,0.0,2.0
177,178.0,Mawile,1d,GA,0.0,2.0
126,127.0,Jynx,1d,GA,0.0,1.0
153,154.0,Hitmonlee,1d,GA,0.0,1.0
207,208.0,Eevee,1d,GA,0.0,1.0
20,21.0,Exeggcute,1d,GA,1.0,1.0
61,62.0,Tentacool,1d,GA,2.0,5.0
0,1.0,Bulbasaur,1d,GA,2.0,4.0
72,73.0,Seaking,1d,GA,2.0,2.0
96,97.0,Magnemite,1d,GA,2.0,2.0


---
AJ needs Venomoth
Megan has Venomoth
They can trade: 


...

---
Aidan needs Dedene
Megan has Dedene
They can trade: 


,ID,Card,Rarity,Pack,Megan has,Aidan extra
240,241.0,Nidoking,1s,GA,2.0,1.0


AJ needs Dedene
Megan has Dedene
They can trade: 


,ID,Card,Rarity,Pack,Megan has,AJ extra
234,235.0,Electrode,1s,GA,2.0,1.0


---
Aidan needs Celebi ex
Megan has Celebi ex
AJ needs Celebi ex
Megan has Celebi ex
---
